In [1]:
import sys
sys.path.insert(0, "../src")

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
import gc
from pathlib import Path
from tqdm.notebook import tqdm
import warnings

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import torchvision.transforms as T
import albumentations as A
from albumentations.pytorch import ToTensorV2

import pytorch_lightning as pl
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
)

import optim
import loss
from utils import visualize, radar2precipitation, seed_everything

<IPython.core.display.Javascript object>

In [4]:
warnings.simplefilter("ignore")

<IPython.core.display.Javascript object>

In [5]:
args = dict(
    seed=42,
    dams=(6071, 6304, 7026, 7629, 7767, 8944, 11107),
    train_folds_csv=Path("../input/train_folds.csv"),
    train_data_path=Path("../input/train"),
    test_data_path=Path("../input/test"),
    model_dir=Path("../models"),
    output_dir=Path("../output"),
    rng=255.0,
    num_workers=4,
    gpus=1,
    lr=5e-4,
    max_epochs=50,
    batch_size=128,
    precision=16,
    optimizer="adamw",
    scheduler="cosine",
    warmup_epochs=1,
    accumulate_grad_batches=1,
    gradient_clip_val=5.0,
)

<IPython.core.display.Javascript object>

# 🔥 Baseline ⚡️

## Sketch

In [6]:
df = pd.read_csv(args["train_folds_csv"])
df.head()

,filename,fold
0,train_60668.npy,0
1,train_60911.npy,0
2,train_14956.npy,0
3,train_24086.npy,0
4,train_09805.npy,0


<IPython.core.display.Javascript object>

In [7]:
fn = df.loc[0, "filename"]
fn

'train_60668.npy'

<IPython.core.display.Javascript object>

In [8]:
path = args["train_data_path"] / fn
path

PosixPath('../input/train/train_60668.npy')

<IPython.core.display.Javascript object>

In [9]:
data = np.load(path)
data.shape

(120, 120, 5)

<IPython.core.display.Javascript object>

In [10]:
tfms = A.Compose(
    [
        A.PadIfNeeded(min_height=128, min_width=128, always_apply=True, p=1),
        A.VerticalFlip(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.Transpose(p=0.5),
        ToTensorV2(always_apply=True, p=1),
    ]
)

<IPython.core.display.Javascript object>

In [11]:
augmented = tfms(image=data)

<IPython.core.display.Javascript object>

In [12]:
image = augmented["image"]
image.shape

torch.Size([5, 128, 128])

<IPython.core.display.Javascript object>

In [13]:
image[0].min(), image[0].max()

(tensor(0, dtype=torch.uint8), tensor(220, dtype=torch.uint8))

<IPython.core.display.Javascript object>

## Dataset

In [14]:
class NowcastingDataset(Dataset):
    def __init__(self, paths, tfms=None, test=False):
        self.paths = paths
        if tfms is not None:
            self.tfms = tfms
        else:
            self.tfms = A.Compose(
                [
                    A.PadIfNeeded(
                        min_height=128, min_width=128, always_apply=True, p=1
                    ),
                    ToTensorV2(always_apply=True, p=1),
                ]
            )
        self.test = test

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        data = np.load(path)

        augmented = self.tfms(image=data)
        data = augmented["image"]

        x = data[:4, :, :]
        x = x / args["rng"]
        if self.test:
            return x
        else:
            y = data[4, :, :]
            y = y / args["rng"]
            y = y.unsqueeze(0)

            return x, y

<IPython.core.display.Javascript object>

In [15]:
class NowcastingDataModule(pl.LightningDataModule):
    def __init__(
        self,
        train_df=None,
        val_df=None,
        batch_size=args["batch_size"],
        num_workers=args["num_workers"],
        test=False,
    ):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.test = test

    def setup(self, stage="train"):
        if stage == "train":
            train_paths = [
                args["train_data_path"] / fn for fn in self.train_df.filename.values
            ]
            val_paths = [
                args["train_data_path"] / fn for fn in self.val_df.filename.values
            ]
            self.train_dataset = NowcastingDataset(train_paths)
            self.val_dataset = NowcastingDataset(val_paths)
        else:
            test_paths = list(sorted(args["test_data_path"].glob("*.npy")))
            self.test_dataset = NowcastingDataset(test_paths, test=True)

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            sampler=RandomSampler(self.train_dataset),
            pin_memory=True,
            num_workers=self.num_workers,
            drop_last=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=2 * self.batch_size,
            sampler=SequentialSampler(self.val_dataset),
            pin_memory=True,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=2 * self.batch_size,
            sampler=SequentialSampler(self.test_dataset),
            pin_memory=True,
            num_workers=self.num_workers,
        )

<IPython.core.display.Javascript object>

In [16]:
# df = pd.read_csv(args["train_folds_csv"])

# fold = 0
# train_df = df[df["fold"] != fold]
# val_df = df[df["fold"] == fold]

# datamodule = NowcastingDataModule(train_df, val_df)
# datamodule.setup()

# for batch in datamodule.train_dataloader():
#     xs, ys = batch
#     idx = np.random.randint(len(xs))
#     x, y = xs[idx], ys[idx]
#     print(x.shape, y.shape)
#     x = x.detach().cpu().numpy().transpose(1, 2, 0)
#     y = y.unsqueeze(-1)
#     y = y.detach().cpu().numpy()
#     visualize(x, y)
#     break

<IPython.core.display.Javascript object>

## Model

### Layers

In [17]:
class Block(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.net(x)


class Encoder(nn.Module):
    def __init__(self, chs=[4, 64, 128]):
        super().__init__()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.blocks = nn.ModuleList(
            [Block(chs[i], chs[i + 1]) for i in range(len(chs) - 1)]
        )
        self.conv = nn.Conv2d(128, 512, kernel_size=3, padding=1)

    def forward(self, x):
        ftrs = []
        for block in self.blocks:
            x = block(x)
            ftrs.append(x)
            x = self.pool(x)
        x = self.conv(x)
        ftrs.append(x)
        return ftrs


class Decoder(nn.Module):
    def __init__(self, chs=[512, 128, 64]):
        super().__init__()
        self.tr_convs = nn.ModuleList(
            [
                nn.ConvTranspose2d(chs[i], chs[i + 1], kernel_size=2, stride=2)
                for i in range(len(chs) - 1)
            ]
        )
        self.blocks = nn.ModuleList(
            [Block(2 * chs[i + 1], chs[i + 1]) for i in range(len(chs) - 1)]
        )

    def forward(self, x, ftrs):
        for i, ftr in enumerate(ftrs):
            x = self.tr_convs[i](x)
            x = torch.cat([ftr, x], dim=1)
            x = self.blocks[i](x)
        return x

<IPython.core.display.Javascript object>

### Model

In [18]:
class Baseline(pl.LightningModule):
    def __init__(
        self,
        lr=args["lr"],
        enc_chs=[4, 64, 128],
        dec_chs=[512, 128, 64],
        num_train_steps=None,
    ):
        super().__init__()
        self.lr = lr
        self.num_train_steps = num_train_steps
        self.criterion = nn.L1Loss()
        self.encoder = Encoder(enc_chs)
        self.decoder = Decoder(dec_chs)
        self.out = nn.Sequential(
            nn.Conv2d(64, 1, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        ftrs = self.encoder(x)
        ftrs = ftrs[::-1]
        x = self.decoder(ftrs[0], ftrs[1:])
        out = self.out(x)
        return out

    def shared_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        return loss, y, y_hat

    def training_step(self, batch, batch_idx):
        loss, y, y_hat = self.shared_step(batch, batch_idx)
        self.log("train_loss", loss)

        for i, param_group in enumerate(self.optimizer.param_groups):
            self.log(f"lr/lr{i}", param_group["lr"])

        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        loss, y, y_hat = self.shared_step(batch, batch_idx)

        return {"loss": loss, "y": y.detach(), "y_hat": y_hat.detach()}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
        self.log("val_loss", avg_loss)

        y = torch.cat([x["y"] for x in outputs])
        y_hat = torch.cat([x["y_hat"] for x in outputs])

        crop = T.CenterCrop(120)
        y = crop(y)
        y_hat = crop(y_hat)

        batch_size = len(y)
        y = y.reshape(batch_size, -1)
        y = y.detach().cpu().numpy()
        y *= args["rng"]
        y_hat = y_hat.reshape(batch_size, -1)
        y_hat = y_hat.detach().cpu().numpy()
        y_hat *= args["rng"]

        y = y[:, args["dams"]]
        y_hat = y_hat[:, args["dams"]]

        y_true = radar2precipitation(y)
        y_true = np.where(y_true >= 0.1, 1.0, 0.0)
        y_pred = radar2precipitation(y_hat)
        y_pred = np.where(y_pred >= 0.1, 1.0, 0.0)

        y_true = y_true.ravel()
        y_pred = y_pred.ravel()

        tn, fp, fn, tp = metrics.confusion_matrix(y_true, y_pred).ravel()
        csi = tp / (tp + tn + tp)
        self.log("csi", csi)

        mae = metrics.mean_absolute_error(y, y_hat, sample_weight=y_true)
        self.log("mae", mae)

        comp_metric = mae / (csi + 1e-12)
        self.log("comp_metric", comp_metric)

        print(
            f"Epoch {self.current_epoch} | MAE/CSI: {comp_metric} | MAE: {mae} | CSI: {csi} | Loss: {avg_loss}"
        )

    def configure_optimizers(self):
        if args["optimizer"] == "adam":
            self.optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        elif args["optimizer"] == "adamw":
            self.optimizer = AdamW(self.parameters(), lr=self.lr)
        elif args["optimizer"] == "radam":
            self.optimizer = optim.RAdam(self.parameters(), lr=self.lr)
        elif args["optimizer"] == "ranger":
            self.optimizer = optim.RAdam(self.parameters(), lr=self.lr)
            self.optimizer = optim.Lookahead(self.optimizer)

        if args["scheduler"] == "cosine":
            self.scheduler = get_cosine_schedule_with_warmup(
                self.optimizer,
                num_warmup_steps=self.num_train_steps * args["warmup_epochs"],
                num_training_steps=self.num_train_steps * args["max_epochs"],
            )
            return [self.optimizer], [{"scheduler": self.scheduler, "interval": "step"}]
        elif args["scheduler"] == "step":
            self.scheduler = torch.optim.lr_scheduler.StepLR(
                self.optimizer, step_size=10, gamma=0.5
            )
            return [self.optimizer], [
                {"scheduler": self.scheduler, "interval": "epoch"}
            ]
        elif args["scheduler"] == "plateau":
            self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer, mode="min", factor=0.1, patience=3, verbose=True
            )
            return [self.optimizer], [
                {
                    "scheduler": self.scheduler,
                    "interval": "epoch",
                    "reduce_on_plateau": True,
                    "monitor": "comp_metric",
                }
            ]
        else:
            self.scheduler = None
            return [self.optimizer]

<IPython.core.display.Javascript object>

## Train

In [19]:
seed_everything(args["seed"])
pl.seed_everything(args["seed"])

42

<IPython.core.display.Javascript object>

In [20]:
df = pd.read_csv(args["train_folds_csv"])

<IPython.core.display.Javascript object>

In [14]:
def train_fold(df, fold):
    train_df = df[df.fold != fold]
    val_df = df[df.fold == fold]

    datamodule = NowcastingDataModule(train_df, val_df)
    datamodule.setup()

    num_train_steps = np.ceil(
        len(train_df) // args["batch_size"] / args["accumulate_grad_batches"]
    )
    model = Baseline(num_train_steps=num_train_steps)

    trainer = pl.Trainer(
        gpus=args["gpus"],
        max_epochs=args["max_epochs"],
        precision=args["precision"],
        progress_bar_refresh_rate=50,
        benchmark=True,
    )

    print(f"Training fold {fold}...")
    trainer.fit(model, datamodule)
    
    checkpoint = (
        args["model_dir"]
        / f"baseline_fold{fold}_bs{args['batch_size']}_epochs{args['max_epochs']}_lr{model.lr}_{args['optimizer']}.ckpt"
    )
    trainer.save_checkpoint(checkpoint)
    print("Model saved at", checkpoint)
    
    del model, trainer, datamodule
    gc.collect()
    torch.cuda.empty_cache()    

<IPython.core.display.Javascript object>

In [14]:
# AdamW
fold = 0
train_fold(df, fold)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Training fold0...



  | Name      | Type       | Params
-----------------------------------------
0 | criterion | L1Loss     | 0     
1 | encoder   | Encoder    | 666 K 
2 | decoder   | Decoder    | 664 K 
3 | out       | Sequential | 577   


Epoch 0 | MAE/CSI: 1000000000000.0 | MAE: 1.0 | CSI: 0.0 | Loss: 0.07034780830144882


Epoch 0 | MAE/CSI: 0.8953259817369302 | MAE: 0.10786948176583493 | CSI: 0.12048067850736674 | Loss: 0.014457848854362965


...

Epoch 28 | MAE/CSI: 0.8923343991380076 | MAE: 0.10687140115163148 | CSI: 0.11976608909616909 | Loss: 0.010615041479468346


../src/utils.py:41: RuntimeWarning: overflow encountered in multiply
  z *= np.power(10.0, dbz_max / 10.0)


Epoch 29 | MAE/CSI: 0.8853361925398303 | MAE: 0.10610364683301343 | CSI: 0.11984559958826557 | Loss: 0.01060927752405405

Model saved at ../models/baseline_bs128_epochs30_lr0.0005_adamw.ckpt


../src/utils.py:41: RuntimeWarning: overflow encountered in multiply
  z *= np.power(10.0, dbz_max / 10.0)
/home/isleof/.pyenv/versions/miniconda3-latest/envs/torch/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


<IPython.core.display.Javascript object>

In [13]:
# RAdam
fold = 0
train_fold(df, fold)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Training fold0...



  | Name      | Type       | Params
-----------------------------------------
0 | criterion | L1Loss     | 0     
1 | encoder   | Encoder    | 666 K 
2 | decoder   | Decoder    | 664 K 
3 | out       | Sequential | 577   


Epoch 0 | MAE/CSI: 1000000000000.0 | MAE: 1.0 | CSI: 0.0 | Loss: 0.07034780830144882


../src/optim/radam.py:49: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


...

Epoch 28 | MAE/CSI: 0.8978677068424368 | MAE: 0.10748560460652591 | CSI: 0.11971207315566174 | Loss: 0.010626324452459812


../src/utils.py:41: RuntimeWarning: overflow encountered in multiply
  z *= np.power(10.0, dbz_max / 10.0)


Epoch 29 | MAE/CSI: 0.8928407437743435 | MAE: 0.10694817658349329 | CSI: 0.11978415784487374 | Loss: 0.010623933747410774

Model saved at ../models/baseline_bs128_epochs30_lr0.0005_radam.ckpt


../src/utils.py:41: RuntimeWarning: overflow encountered in multiply
  z *= np.power(10.0, dbz_max / 10.0)
/home/isleof/.pyenv/versions/miniconda3-latest/envs/torch/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


<IPython.core.display.Javascript object>

In [14]:
# Ranger
fold = 0
train_fold(df, fold)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Training fold0...



  | Name      | Type       | Params
-----------------------------------------
0 | criterion | L1Loss     | 0     
1 | encoder   | Encoder    | 666 K 
2 | decoder   | Decoder    | 664 K 
3 | out       | Sequential | 577   


Epoch 0 | MAE/CSI: 1000000000000.0 | MAE: 1.0 | CSI: 0.0 | Loss: 0.07034780830144882


Epoch 0 | MAE/CSI: 1.1385640966037704 | MAE: 0.13389635316698656 | CSI: 0.11760106748952318 | Loss: 0.01573796570301056


...

Epoch 26 | MAE/CSI: 0.9082203937558084 | MAE: 0.10863723608445297 | CSI: 0.11961549953122264 | Loss: 0.010760427452623844


Epoch 27 | MAE/CSI: 0.9217818510578139 | MAE: 0.11009596928982726 | CSI: 0.11943820456278466 | Loss: 0.010757229290902615


Epoch 28 | MAE/CSI: 0.9232172476669039 | MAE: 0.11024952015355087 | CSI: 0.11941882631768767 | Loss: 0.010758249089121819


Epoch 29 | MAE/CSI: 0.9124960947270578 | MAE: 0.1090978886756238 | CSI: 0.11955984174085064 | Loss: 0.010757893323898315

Model saved at ../models/baseline_bs128_epochs30_lr0.0005_ranger.ckpt


<IPython.core.display.Javascript object>

In [16]:
# AdamW bs50
fold = 0
train_fold(df, fold)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Training fold0...



  | Name      | Type       | Params
-----------------------------------------
0 | criterion | L1Loss     | 0     
1 | encoder   | Encoder    | 666 K 
2 | decoder   | Decoder    | 664 K 
3 | out       | Sequential | 577   


Epoch 0 | MAE/CSI: 1000000000000.0 | MAE: 1.0 | CSI: 0.0 | Loss: 0.07034780830144882


Epoch 0 | MAE/CSI: 1.431734138641048 | MAE: 0.16314779270633398 | CSI: 0.11395117871517432 | Loss: 0.014841477386653423


...

Epoch 46 | MAE/CSI: 0.8986140990898661 | MAE: 0.10756238003838772 | CSI: 0.11969807745775453 | Loss: 0.010502759367227554


Epoch 47 | MAE/CSI: 0.8943869105405711 | MAE: 0.1071017274472169 | CSI: 0.11974876441515651 | Loss: 0.010501205921173096


Epoch 48 | MAE/CSI: 0.8986696199329683 | MAE: 0.10756238003838772 | CSI: 0.11969068237280805 | Loss: 0.010503779165446758


Epoch 49 | MAE/CSI: 0.8781667057234775 | MAE: 0.10533589251439539 | CSI: 0.11994976788232509 | Loss: 0.010497772134840488

Model saved at ../models/baseline_bs128_epochs50_lr0.0005_adamw.ckpt


<IPython.core.display.Javascript object>

In [14]:
# AdamW bs50 lr 1e-4
fold = 0
train_fold(df, fold)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Training fold0...



  | Name      | Type       | Params
-----------------------------------------
0 | criterion | L1Loss     | 0     
1 | encoder   | Encoder    | 666 K 
2 | decoder   | Decoder    | 664 K 
3 | out       | Sequential | 577   


Epoch 0 | MAE/CSI: 1000000000000.0 | MAE: 1.0 | CSI: 0.0 | Loss: 0.07034780830144882


Epoch 0 | MAE/CSI: 1.1213412779907173 | MAE: 0.13205374280230325 | CSI: 0.1177640967946915 | Loss: 0.015904391184449196


...

Epoch 46 | MAE/CSI: 0.933066581537057 | MAE: 0.11132437619961612 | CSI: 0.11931021687144389 | Loss: 0.010794575326144695


Epoch 47 | MAE/CSI: 0.9401394272301328 | MAE: 0.11209213051823416 | CSI: 0.11922926245901977 | Loss: 0.010793674737215042


Epoch 48 | MAE/CSI: 0.9387956837220837 | MAE: 0.11193857965451055 | CSI: 0.11923635951303488 | Loss: 0.01079593040049076


Epoch 49 | MAE/CSI: 0.9216488734398317 | MAE: 0.11009596928982726 | CSI: 0.1194554373814824 | Loss: 0.010792912915349007

Model saved at ../models/baseline_bs128_epochs50_lr0.0001_adamw.ckpt


<IPython.core.display.Javascript object>

In [16]:
# AdamW bs50 lr 1e-3
fold = 0
train_fold(df, fold)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Training fold0...



  | Name      | Type       | Params
-----------------------------------------
0 | criterion | L1Loss     | 0     
1 | encoder   | Encoder    | 666 K 
2 | decoder   | Decoder    | 664 K 
3 | out       | Sequential | 577   


Epoch 0 | MAE/CSI: 1000000000000.0 | MAE: 1.0 | CSI: 0.0 | Loss: 0.07034780830144882


Epoch 0 | MAE/CSI: 1.138320426835042 | MAE: 0.13381957773512476 | CSI: 0.11755879502756099 | Loss: 0.01589236781001091


...

Epoch 46 | MAE/CSI: 0.9281252628678694 | MAE: 0.1107869481765835 | CSI: 0.11936637500128827 | Loss: 0.010795307345688343


Epoch 47 | MAE/CSI: 0.9366597046688451 | MAE: 0.11170825335892515 | CSI: 0.11926236689928153 | Loss: 0.010794851928949356


Epoch 48 | MAE/CSI: 0.9374134629679352 | MAE: 0.11178502879078694 | CSI: 0.11924837140265523 | Loss: 0.010798281989991665


Epoch 49 | MAE/CSI: 0.921677368643685 | MAE: 0.11009596928982726 | CSI: 0.11945174421600453 | Loss: 0.010794124566018581

Model saved at ../models/baseline_bs128_epochs50_lr0.0001_adamw.ckpt


<IPython.core.display.Javascript object>

In [ ]:
# AdamW bs50 lr 5e-4
fold = 0
train_fold(df, fold)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Training fold0...



  | Name      | Type       | Params
-----------------------------------------
0 | criterion | L1Loss     | 0     
1 | encoder   | Encoder    | 666 K 
2 | decoder   | Decoder    | 664 K 
3 | out       | Sequential | 577   


Epoch 0 | MAE/CSI: 1000000000000.0 | MAE: 1.0 | CSI: 0.0 | Loss: 0.07034783065319061


Epoch 0 | MAE/CSI: 0.9381335799455521 | MAE: 0.11247600767754319 | CSI: 0.11989338200976986 | Loss: 0.013658225536346436


...

Epoch 43 | MAE/CSI: 0.8783836508380664 | MAE: 0.10533589251439539 | CSI: 0.11992014242639416 | Loss: 0.010503781028091908


Epoch 44 | MAE/CSI: 0.8798422581432144 | MAE: 0.105489443378119 | CSI: 0.11989585905985017 | Loss: 0.010508840903639793


Epoch 45 | MAE/CSI: 0.9060126520071495 | MAE: 0.10833013435700575 | CSI: 0.11956801498975632 | Loss: 0.010515524074435234


Epoch 46 | MAE/CSI: 0.8869249315791313 | MAE: 0.10625719769673704 | CSI: 0.11980404869966141 | Loss: 0.01050545647740364


Epoch 47 | MAE/CSI: 0.891837017510536 | MAE: 0.10679462571976968 | CSI: 0.11974679635633781 | Loss: 0.010504990816116333


In [15]:
# AdamW bs50 lr 5e-4
for fold in range(5):
    train_fold(df, fold)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Training fold0...



  | Name      | Type       | Params
-----------------------------------------
0 | criterion | L1Loss     | 0     
1 | encoder   | Encoder    | 666 K 
2 | decoder   | Decoder    | 664 K 
3 | out       | Sequential | 577   


Epoch 0 | MAE/CSI: 1000000000000.0 | MAE: 1.0 | CSI: 0.0 | Loss: 0.07034780830144882


Epoch 0 | MAE/CSI: 0.9437922424302221 | MAE: 0.11309021113243763 | CSI: 0.1198253238872696 | Loss: 0.01358635164797306


...

Epoch 46 | MAE/CSI: 0.8607582781995061 | MAE: 0.10109612806560997 | CSI: 0.11745007933727615 | Loss: 0.01040069479495287


Epoch 47 | MAE/CSI: 0.8755787668255379 | MAE: 0.10267329074994086 | CSI: 0.11726334013479256 | Loss: 0.010401761159300804


Epoch 48 | MAE/CSI: 0.8866706187256753 | MAE: 0.10385616276318903 | CSI: 0.11713048855905998 | Loss: 0.010412335395812988


Epoch 49 | MAE/CSI: 0.8792732161385125 | MAE: 0.10306758142102358 | CSI: 0.11721906175282382 | Loss: 0.010402982123196125

Model saved at ../models/baseline_fold4_bs128_epochs50_lr0.0005_adamw.ckpt


<IPython.core.display.Javascript object>

## Inference

In [21]:
def inference(checkpoints):
    datamodule = NowcastingDataModule()
    datamodule.setup("test")

    test_paths = datamodule.test_dataset.paths
    test_filenames = [path.name for path in test_paths]
    final_preds = np.zeros((len(datamodule.test_dataset), 14400))

    for checkpoint in checkpoints:
        print("Inference from", checkpoint)
        model = Baseline.load_from_checkpoint(str(checkpoint))
        model.cuda()
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in tqdm(datamodule.test_dataloader()):
                batch = batch.cuda()
                imgs = model(batch)
                imgs = imgs.detach().cpu().numpy()
                imgs = imgs[:, 0, 4:124, 4:124]
                imgs = args["rng"] * imgs
                imgs = imgs.clip(0, 255)
                imgs = imgs.round()
                preds.append(imgs)

        preds = np.concatenate(preds)
        preds = preds.astype(np.uint8)
        preds = preds.reshape(-1, 14400)
        final_preds += preds / len(checkpoints)

        del model
        gc.collect()
        torch.cuda.empty_cache()

    final_preds = final_preds.round()
    final_preds = final_preds.astype(np.uint8)

    subm = pd.DataFrame()
    subm["file_name"] = test_filenames
    for i in tqdm(range(14400)):
        subm[str(i)] = final_preds[:, i]

    return subm

<IPython.core.display.Javascript object>

In [22]:
checkpoints = [
    args["model_dir"]
    / f"baseline_fold{fold}_bs{args['batch_size']}_epochs{args['max_epochs']}_lr{args['lr']}_{args['optimizer']}.ckpt"
    for fold in range(5)
]

<IPython.core.display.Javascript object>

In [23]:
subm = inference(checkpoints)

Inference from ../models/baseline_fold0_bs128_epochs50_lr0.0005_adamw.ckpt



Inference from ../models/baseline_fold1_bs128_epochs50_lr0.0005_adamw.ckpt



Inference from ../models/baseline_fold2_bs128_epochs50_lr0.0005_adamw.ckpt



Inference from ../models/baseline_fold3_bs128_epochs50_lr0.0005_adamw.ckpt



Inference from ../models/baseline_fold4_bs128_epochs50_lr0.0005_adamw.ckpt


<IPython.core.display.Javascript object>

In [24]:
output_csv = (
    args["output_dir"]
    / f"baseline_bs{args['batch_size']}_epochs{args['max_epochs']}_lr{args['lr']}_{args['optimizer']}.csv"
)
subm.to_csv(output_csv, index=False)
subm.head()

,file_name,0,1,2,3,4,5,6,7,8,...,14390,14391,14392,14393,14394,14395,14396,14397,14398,14399
0,test_00000.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,test_00001.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,test_00002.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,test_00003.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,test_00004.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>